In [1]:

  # Import necessary libraries
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import dash_leaflet as dl
import pandas as pd
from pymongo import MongoClient
from animal_shelter import AnimalShelter  





In [2]:

# Load CSV data using pandas
csv_file = 'aac_shelter_outcomes.csv'  # Make sure your CSV file is in the same directory
df = pd.read_csv(csv_file)

# Optionally drop unnecessary columns for cleaner data visualization
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Preview the first few rows to ensure data is loaded correctly
df.head()




,Unnamed: 0,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome,location_lat,location_long,age_upon_outcome_in_weeks
0,1,3 years,A746874,Cat,Domestic Shorthair Mix,Black/White,2014-04-10,2017-04-11 09:00:00,2017-04-11T09:00:00,NaN,SCRP,Transfer,Neutered Male,30.506658,-97.340878,156.767857
1,2,1 year,A725717,Cat,Domestic Shorthair Mix,Silver Tabby,2015-05-02,2016-05-06 10:49:00,2016-05-06T10:49:00,NaN,SCRP,Transfer,Spayed Female,30.652598,-97.741996,52.921528
2,3,2 years,A716330,Dog,Chihuahua Shorthair Mix,Brown/White,2013-11-18,2015-12-28 18:43:00,2015-12-28T18:43:00,Frank,NaN,Adoption,Neutered Male,30.759575,-97.552375,110.111409
3,4,7 months,A733653,Cat,Siamese Mix,Seal Point,2016-01-25,2016-08-27 18:11:00,2016-08-27T18:11:00,Kitty,NaN,Adoption,Intact Female,30.318806,-97.724038,30.822520
4,5,2 years,A691584,Dog,Labrador Retriever Mix,Tan/White,2012-11-06,2015-05-30 13:48:00,2015-05-30T13:48:00,Luke,NaN,Return to Owner,Neutered Male,30.710482,-97.562297,133.653571


In [3]:
# Initialize the Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout of the dashboard
app.layout = dbc.Container([
    # Logo and Title
    dbc.Row([
        dbc.Col(html.Img(src='/assets/Grazioso.png', style={'height': '150px'}), width=12, className='text-center'),  # Updated logo path
    ], className='mb-4'),

    dbc.Row([
        dbc.Col(html.H1("Grazioso Salvare - Dog Rescue Dashboard", className='text-center text-primary mb-2'), width=12),
        dbc.Col(html.H2("Dheeraj Kollapaneni", className='text-center mb-4'), width=12),
    ]),

    # Rescue type filter
    dbc.Row([
        dbc.Col([
            html.Label("Filter by Rescue Type:"),
            dcc.RadioItems(
                id='rescue-type-radio',
                options=[
                    {'label': 'All (Reset)', 'value': 'Reset'},
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
                ],
                value='Reset',  # Default selection
                labelStyle={'display': 'inline-block', 'margin-right': '15px'}
            )
        ], width=12, className='mb-4')
    ]),

    # Full-width data table
    dbc.Row([
        dbc.Col([
            dash_table.DataTable(
                id='datatable-id',  
                columns=[{'name': i, 'id': i} for i in df.columns],
                data=df.to_dict('records'),
                sort_action='native',
                filter_action='native',
                row_selectable='single',  # Enable single row selection
                page_size=10,
                style_table={'height': '350px', 'overflowY': 'auto', 'width': '100%'},
                style_cell={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'textAlign': 'left'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                }
            )
        ], width=12)  # Full width table
    ], className='mb-5'),

    # Row for pie chart and map (side by side)
    dbc.Row([
        # Pie chart
        dbc.Col([
            html.H5("Breed Distribution", className='text-center mb-3'),
            dcc.Graph(id='pie-chart-id', style={'height': '350px'})  # Pie chart
        ], width=6),

        # Map for geolocation
        dbc.Col([
            html.H5("Animal Location", className='text-center mb-3'),
            html.Div(id='map-id', style={'height': '350px'})  # Map for geolocation
        ], width=6)
    ], align="center")
], fluid=True)


In [4]:

@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-radio', 'value')]
)
def update_table(selected_rescue_type):
    if selected_rescue_type == "Water Rescue":
        filtered_df = df[df['breed'].isin(["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"])]
    elif selected_rescue_type == "Mountain or Wilderness Rescue":
        filtered_df = df[df['breed'].isin(["German Shepherd", "Alaskan Malamute", "Bloodhound", "Golden Retriever"])]
    elif selected_rescue_type == "Disaster or Individual Tracking":
        filtered_df = df[df['breed'].isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever"])]
    else:
        filtered_df = df  # Show all data if "Reset" is selected

    return filtered_df.to_dict('records')


In [5]:
@app.callback(
    Output('pie-chart-id', 'figure'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_pie_chart(derived_virtual_data, selected_rows):
    # Create a DataFrame from the table's current state (filtered or full)
    filtered_df = pd.DataFrame(derived_virtual_data)

    if selected_rows is not None and len(selected_rows) > 0:
        # If a row is selected, get the breed of that row and filter by it
        selected_row = selected_rows[0]
        selected_breed = filtered_df.iloc[selected_row]['breed']
        selected_data = filtered_df[filtered_df['breed'] == selected_breed]

        # Display the distribution for the selected breed only
        fig = px.pie(selected_data, names='breed', title=f'Breed Distribution for {selected_breed}')
    else:
        # If no row is selected, show overall distribution based on filtered data
        fig = px.pie(filtered_df, names='breed', title='Overall Breed Distribution')

    fig.update_layout(margin=dict(t=50, b=50, l=50, r=50), height=400)
    return fig



In [6]:
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'derived_virtual_data')]
)
def update_map(selected_rows, data):
    if selected_rows is not None and len(selected_rows) > 0:
        row = selected_rows[0]
        selected_df = pd.DataFrame(data)
        lat = selected_df.iloc[row]['location_lat']
        lon = selected_df.iloc[row]['location_long']
        
        return dl.Map(style={'width': '100%', 'height': '100%'}, center=[lat, lon], zoom=12, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(selected_df.iloc[row]['name']),
                dl.Popup([html.H1("Animal Location"), html.P(selected_df.iloc[row]['breed'])])
            ])
        ])
    return dl.Map(style={'width': '100%', 'height': '100%'}, center=[30.2672, -97.7431], zoom=10, children=[dl.TileLayer(id="base-layer-id")])


In [7]:
# Run the app
app.run_server(debug=True)



Dash app running on http://127.0.0.1:21662/
